# Cerebellar Model Articulation Controller(CMAC)


## 概论
小脑模型的初始设想很简单，希望设计一个这样的模型：

1. **足够快**
2. **拟合**

### 足够快
对于第一点，传统的神经网络均是使用浮点数进行计算，使用浮点数显然存在两个缺点：其一是占用空间大，其二是运算不够快。但也存在非常明显的优点：计算精度高。

若为了在不太降低精度的条件下尽可能提高模型运算效率，显然有两个角度：其一是改进模型，其二是改变数值存储方式。

量化技术就是这样一类通过改变数值存储方式提高模型运算效率的方式。对于现代神经网络，训练通常采用32位浮点数，推理时则可以选用16位浮点数以提高精度，而当部署到边缘设备上时则可以进一步采用**int8量化技术**。

对于CMAC，同样的，为了提升运算效率，初始数据输入时会进行量化。此外，为了进一步提升效率，CMAC还引入了**哈希散列**，通过查表的方式将相近的输入映射到相似的地址上。

哈希散列同时也引入了不确定的因素。哈希散列是一种压缩映射，其很有可能将不同的输入映射到同一个地址，即导致了**碰撞**。从另一个角度讲，这也引入了非线性变换，即在原空间可能相隔非常远的两个输入，在映射后可能十分相近甚至发生**碰撞**。


### 拟合
为了实现拟合，CMAC在查表后建立了一个自适应线性层，实现地址到输出的线性估计
由于采用了哈希，实际上建立了输入与地址的映射表。在不考虑碰撞的情况下，一个特定的输入会激活特定的地址，而特定的地址会激活特定的自适应线性层的输入单元，这些单元则会连接到输出。

不同于传统的神经网络，进行推理时同层所有的神经元均会参与运算，CMAC中仅有被激活的输入单元才会参与运算，这显然也加速了CMAC的运算速度。


## 符号定义

### 空间
|符号|含义|
|:-:|:-:|
|$S$|输入空间|
|$M$|扩充地址空间|
|$MC$|扩充地址空间长度|
|$A_c$|虚拟存储空间|
|$A_p$|实际存储空间|
|$F$|输出空间|

### 数据
|符号|含义|
|:-:|:-:|
|$\bm{s}$|输入向量|
|$\bm{m}$|扩充后矩阵|
|$\bm{a}$|虚拟存储空间向量|
|$\bm{d}$|实际存储空间向量|
|$\bm{\hat{y}}$|预测输出|
|$\bm{y}$|真实输出|

### 参数
|符号|含义|
|:-:|:-:|
|s|输入空间维度|
|q|量化级|
|c|扩充地址维度|
|$\bm{W}$|输出权值矩阵|


## 正向运算

CMAC的整体流程如下：

* 输入空间$S$离散化
* 输入空间$S$ $\rightarrow$ 扩充地址空间$M$
* 扩充地址空间$M$ $\rightarrow$ 虚拟存储空间$A_c$
* 虚拟存储空间$A_c$ $\rightarrow$ 实际存储空间$A_p$
* 实际存储空间$A_p$ $\rightarrow$ 输出空间$F$

第零步为离散化。一方面是加速运算，另一方面也是配合后续的Hash

第一步是在进行升维

第二步是将第一步中升维到高维的多个分量组合为一个向量

第三步为Hash

第四步为自适应线性拟合

### 离散化

输入为$\bm{s}=[s_1, s_2, \cdots, s_s]$

设定第n个维度的取值范围为$[n_{min}, n_{max}]$，量化级为q_n

则第n个维度的离散值为
$$
\begin{equation}
    s_n = \lceil\frac{(s_n-n_{min})}{(n_{max}-n_{min})}*q_n\rceil
\end{equation}
$$

### 输入空间到扩充地址空间

每一个输入分量均扩充到c维

对于一个特定的输入分量$s_n$，有对应的扩充后向量$\bm{m_n}$

扩充后向量按照如下的方式进行运算

定义如下的取余运算
$$
\begin{equation}
    \Psi(s_n) = mod(\frac{s_n-1}{c})+1
\end{equation}
$$
则$\bm{m_n}$的第$\Psi(s_n)$位为$s_n$，其他位依次推出

|$\bm{m_{n1}}$|$\bm{m_{n2}}$|$\cdots$|$\bm{m_{n\Psi(s_n)}}$|$\cdots$|$\bm{m_{nc}}$|
|:-:|:-:|:-:|:-:|:-:|:-:|
|$s_n+(c-\Psi(s_n)+1)$|$s_n+(c-\Psi(s_n)+2)$|$\cdots$|$s_n$|$\cdots$|$s_n+(c-\Psi(s_n))$|

### 扩充地址空间到虚拟存储空间
